# 1. 데이터 불러오기

In [212]:
import pandas as pd
import os

pd.options.display.float_format = '{:.3f}'.format
pd.set_option('display.max_columns', None)

PATH = './data/'

file_list= os.listdir(PATH)
csv_list = list()

for file in file_list:
    if file.split(".")[-1] == 'csv':
        csv_list.append(file)
data = ['living_popul', 'resident_popul', 'area', 'store', 'working_popul', 'facilitie', 'estimated_sales']

for name, file in zip(data, csv_list):
    globals()[name] = pd.read_csv(PATH + file, encoding = "ansi")

store = store.drop("Unnamed: 0", axis = 1)
estimated_sales = estimated_sales.drop("Unnamed: 0", axis = 1)

# 2. 데이터 전처리

## 2-0. 시점 통일

In [213]:
years = [20223, 20224, 20231, 20232]

living_popul = living_popul[living_popul['기준_년분기_코드'].isin(years)]
resident_popul = resident_popul[resident_popul['기준_년분기_코드'].isin(years)]
store = store[store['기준_년분기_코드'].isin(years)]
working_popul = working_popul[working_popul['기준_년분기_코드'].isin(years)]
facilitie = facilitie[facilitie['기준_년분기_코드'].isin(years)]
estimated_sales = estimated_sales[estimated_sales['기준_년분기_코드'].isin(years)]

## 2-1. 길단위인구 전처리

### 2-1-1 변수 처리

In [214]:
# 파생변수 생성
living_popul['시간대_21_06_유동인구_수'] = living_popul['시간대_00_06_유동인구_수'] + living_popul['시간대_21_24_유동인구_수']
living_popul['주중_유동인구_수'] = living_popul['월요일_유동인구_수'] + living_popul['화요일_유동인구_수'] + living_popul['수요일_유동인구_수'] + living_popul['목요일_유동인구_수'] + living_popul['금요일_유동인구_수']
living_popul['주말_유동인구_수'] = living_popul['토요일_유동인구_수'] + living_popul['일요일_유동인구_수']
living_popul['연령대_30_40_유동인구_수'] = living_popul['연령대_30_유동인구_수'] + living_popul['연령대_40_유동인구_수']
living_popul['연령대_50_60_유동인구_수'] = living_popul['연령대_50_유동인구_수'] + living_popul['연령대_60_이상_유동인구_수']

# 불필요한 변수 제거
living_popul = living_popul.drop(['월요일_유동인구_수', '화요일_유동인구_수', '수요일_유동인구_수', '목요일_유동인구_수', '금요일_유동인구_수', '토요일_유동인구_수', '일요일_유동인구_수', 
                                  '시간대_00_06_유동인구_수', '시간대_21_24_유동인구_수',
                                  '연령대_30_유동인구_수','연령대_40_유동인구_수','연령대_50_유동인구_수','연령대_60_이상_유동인구_수'], axis = 1)

### 2-1-2. 결측치/이상치/중복

In [215]:
# 결측값 확인
living_popul.isnull().sum()

기준_년분기_코드           0
상권_구분_코드_명          0
상권_코드_명             0
총_유동인구_수            0
남성_유동인구_수           0
여성_유동인구_수           0
연령대_10_유동인구_수       0
연령대_20_유동인구_수       0
시간대_06_11_유동인구_수    0
시간대_11_14_유동인구_수    0
시간대_14_17_유동인구_수    0
시간대_17_21_유동인구_수    0
시간대_21_06_유동인구_수    0
주중_유동인구_수           0
주말_유동인구_수           0
연령대_30_40_유동인구_수    0
연령대_50_60_유동인구_수    0
dtype: int64

In [216]:
# 통계량 확인
living_popul.describe()

,기준_년분기_코드,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,연령대_10_유동인구_수,연령대_20_유동인구_수,시간대_06_11_유동인구_수,시간대_11_14_유동인구_수,시간대_14_17_유동인구_수,시간대_17_21_유동인구_수,시간대_21_06_유동인구_수,주중_유동인구_수,주말_유동인구_수,연령대_30_40_유동인구_수,연령대_50_60_유동인구_수
count,6597.000,6597.000,6597.000,6597.000,6597.000,6597.000,6597.000,6597.000,6597.000,6597.000,6597.000,6597.000,6597.000,6597.000,6597.000
mean,20227.499,831723.936,396454.429,435269.521,106249.107,148852.466,169196.692,107398.432,108468.099,144295.668,302365.121,600110.514,231613.502,281556.790,295065.647
std,4.032,900980.836,437696.274,466905.782,113447.145,216584.734,179874.328,134146.790,137040.744,167412.286,321245.522,660890.833,246084.966,332193.428,308808.842
min,20223.000,12.000,12.000,0.000,0.000,0.000,4.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,20223.000,221703.000,106884.000,113949.000,25620.000,32184.000,45807.000,28462.000,28587.000,38158.000,75590.000,159214.000,60098.000,74298.000,79315.000
50%,20224.000,568553.000,269829.000,297045.000,71540.000,82940.000,117002.000,72028.000,72359.000,96289.000,205750.000,406266.000,159173.000,185168.000,206051.000
75%,20231.000,1138476.000,537610.000,593960.000,147393.000,184525.000,234575.000,140269.000,140608.000,194982.000,417273.000,828500.000,314287.000,370667.000,412023.000
max,20232.000,8657826.000,4693188.000,4032096.000,829062.000,3487417.000,1678321.000,1853219.000,1861965.000,1793125.000,2942275.000,6833618.000,2124591.000,3506912.000,3565244.000


In [217]:
# 중복 확인
living_popul[living_popul.duplicated()]

,기준_년분기_코드,상권_구분_코드_명,상권_코드_명,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,연령대_10_유동인구_수,연령대_20_유동인구_수,시간대_06_11_유동인구_수,시간대_11_14_유동인구_수,시간대_14_17_유동인구_수,시간대_17_21_유동인구_수,시간대_21_06_유동인구_수,주중_유동인구_수,주말_유동인구_수,연령대_30_40_유동인구_수,연령대_50_60_유동인구_수


In [218]:
# 하위 5% 삭제
living_popul = living_popul[living_popul['총_유동인구_수']>living_popul['총_유동인구_수'].quantile(0.05)]

### 2-1-3. 분기별 데이터 연도 기준 변환

In [219]:
living_popul_year = living_popul.groupby(['상권_구분_코드_명','상권_코드_명'],as_index = False)\
                                .mean()\
                                .drop('기준_년분기_코드', axis = 1)

In [220]:
living_popul_year.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1575 entries, 0 to 1574
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   상권_구분_코드_명        1575 non-null   object 
 1   상권_코드_명           1575 non-null   object 
 2   총_유동인구_수          1575 non-null   float64
 3   남성_유동인구_수         1575 non-null   float64
 4   여성_유동인구_수         1575 non-null   float64
 5   연령대_10_유동인구_수     1575 non-null   float64
 6   연령대_20_유동인구_수     1575 non-null   float64
 7   시간대_06_11_유동인구_수  1575 non-null   float64
 8   시간대_11_14_유동인구_수  1575 non-null   float64
 9   시간대_14_17_유동인구_수  1575 non-null   float64
 10  시간대_17_21_유동인구_수  1575 non-null   float64
 11  시간대_21_06_유동인구_수  1575 non-null   float64
 12  주중_유동인구_수         1575 non-null   float64
 13  주말_유동인구_수         1575 non-null   float64
 14  연령대_30_40_유동인구_수  1575 non-null   float64
 15  연령대_50_60_유동인구_수  1575 non-null   float64
dtypes: float64(14), object(2)
memory usage: 19

## 2-2. 상주인구 전처리

### 2-2-1. 변수 처리

In [221]:
# 파생변수 생성
resident_popul['연령대_30_40_상주인구_수'] = resident_popul['연령대_30_상주인구_수']+resident_popul['연령대_40_상주인구_수']
resident_popul['연령대_50_60_상주인구_수'] = resident_popul['연령대_50_상주인구_수']+resident_popul['연령대_60_이상_상주인구_수']

# 불필요한 변수 제거
resident_popul = resident_popul[['기준_년분기_코드', '상권_구분_코드_명', '상권_코드_명',
                                 '총_상주인구_수', '남성_상주인구_수','여성_상주인구_수', 
                                 '연령대_10_상주인구_수', '연령대_20_상주인구_수', '연령대_30_40_상주인구_수', '연령대_50_60_상주인구_수']]

### 2-2-2. 결측치/이상치/중복

In [222]:
# 결측값 확인
resident_popul.isnull().sum()

기준_년분기_코드           0
상권_구분_코드_명          0
상권_코드_명             0
총_상주인구_수            0
남성_상주인구_수           0
여성_상주인구_수           0
연령대_10_상주인구_수       0
연령대_20_상주인구_수       0
연령대_30_40_상주인구_수    0
연령대_50_60_상주인구_수    0
dtype: int64

In [223]:
# 통계량 확인
resident_popul.describe()

,기준_년분기_코드,총_상주인구_수,남성_상주인구_수,여성_상주인구_수,연령대_10_상주인구_수,연령대_20_상주인구_수,연령대_30_40_상주인구_수,연령대_50_60_상주인구_수
count,6512.000,6512.000,6512.000,6512.000,6512.000,6512.000,6512.000,6512.000
mean,20227.495,2361.003,1166.658,1194.345,230.155,300.171,789.651,1041.026
std,4.032,2349.098,1159.233,1195.283,262.098,340.145,834.359,1031.672
min,20223.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,20223.000,656.750,328.000,329.750,52.000,70.000,207.000,281.000
50%,20224.000,1621.000,811.500,817.000,147.000,195.000,528.000,721.000
75%,20231.000,3306.250,1639.000,1677.000,318.000,413.250,1083.000,1466.000
max,20232.000,21341.000,10459.000,10882.000,3078.000,4457.000,7626.000,8342.000


In [224]:
# 중복 확인
resident_popul[resident_popul.duplicated()]

,기준_년분기_코드,상권_구분_코드_명,상권_코드_명,총_상주인구_수,남성_상주인구_수,여성_상주인구_수,연령대_10_상주인구_수,연령대_20_상주인구_수,연령대_30_40_상주인구_수,연령대_50_60_상주인구_수


### 2-2-3. 분기별 데이터 연도 기준 변환

In [225]:
resident_popul_year = resident_popul[resident_popul['기준_년분기_코드'] == 20232].drop(['기준_년분기_코드'],axis=1)

In [226]:
resident_popul_year.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1625 entries, 19887 to 24485
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   상권_구분_코드_명        1625 non-null   object
 1   상권_코드_명           1625 non-null   object
 2   총_상주인구_수          1625 non-null   int64 
 3   남성_상주인구_수         1625 non-null   int64 
 4   여성_상주인구_수         1625 non-null   int64 
 5   연령대_10_상주인구_수     1625 non-null   int64 
 6   연령대_20_상주인구_수     1625 non-null   int64 
 7   연령대_30_40_상주인구_수  1625 non-null   int64 
 8   연령대_50_60_상주인구_수  1625 non-null   int64 
dtypes: int64(7), object(2)
memory usage: 127.0+ KB


## 2-3. 점포 전처리

### 2-3-1. 변수 처리

In [227]:
store.info()

<class 'pandas.core.frame.DataFrame'>
Index: 304774 entries, 1055496 to 1360269
Data columns (total 14 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   기준_년분기_코드    304774 non-null  int64 
 1   상권_구분_코드     304774 non-null  object
 2   상권_구분_코드_명   304774 non-null  object
 3   상권_코드        304774 non-null  int64 
 4   상권_코드_명      304774 non-null  object
 5   서비스_업종_코드    304774 non-null  object
 6   서비스_업종_코드_명  304774 non-null  object
 7   점포_수         304774 non-null  int64 
 8   유사_업종_점포_수   304774 non-null  int64 
 9   개업_율         304774 non-null  int64 
 10  개업_점포_수      304774 non-null  int64 
 11  폐업_률         304774 non-null  int64 
 12  폐업_점포_수      304774 non-null  int64 
 13  프랜차이즈_점포_수   304774 non-null  int64 
dtypes: int64(9), object(5)
memory usage: 34.9+ MB


In [228]:
store['서비스_업종_대분류'] = store['서비스_업종_코드'].str[:3]

In [229]:
store.info()

<class 'pandas.core.frame.DataFrame'>
Index: 304774 entries, 1055496 to 1360269
Data columns (total 15 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   기준_년분기_코드    304774 non-null  int64 
 1   상권_구분_코드     304774 non-null  object
 2   상권_구분_코드_명   304774 non-null  object
 3   상권_코드        304774 non-null  int64 
 4   상권_코드_명      304774 non-null  object
 5   서비스_업종_코드    304774 non-null  object
 6   서비스_업종_코드_명  304774 non-null  object
 7   점포_수         304774 non-null  int64 
 8   유사_업종_점포_수   304774 non-null  int64 
 9   개업_율         304774 non-null  int64 
 10  개업_점포_수      304774 non-null  int64 
 11  폐업_률         304774 non-null  int64 
 12  폐업_점포_수      304774 non-null  int64 
 13  프랜차이즈_점포_수   304774 non-null  int64 
 14  서비스_업종_대분류   304774 non-null  object
dtypes: int64(9), object(6)
memory usage: 37.2+ MB


### 2-3-1. 결측치/이상치/중복

In [230]:
# 결측값 확인
store.isnull().sum()

기준_년분기_코드      0
상권_구분_코드       0
상권_구분_코드_명     0
상권_코드          0
상권_코드_명        0
서비스_업종_코드      0
서비스_업종_코드_명    0
점포_수           0
유사_업종_점포_수     0
개업_율           0
개업_점포_수        0
폐업_률           0
폐업_점포_수        0
프랜차이즈_점포_수     0
서비스_업종_대분류     0
dtype: int64

In [231]:
# 통계량 확인
store.describe()

,기준_년분기_코드,상권_코드,점포_수,유사_업종_점포_수,개업_율,개업_점포_수,폐업_률,폐업_점포_수,프랜차이즈_점포_수
count,304774.000,304774.000,304774.000,304774.000,304774.000,304774.000,304774.000,304774.000,304774.000
mean,20227.497,3114874.594,6.056,6.506,2.292,0.138,2.479,0.179,0.450
std,4.031,11742.077,42.503,42.878,10.900,0.615,11.805,1.158,2.232
min,20223.000,3001491.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,20223.000,3110436.000,1.000,1.000,0.000,0.000,0.000,0.000,0.000
50%,20224.000,3110871.000,2.000,2.000,0.000,0.000,0.000,0.000,0.000
75%,20231.000,3120141.000,4.000,5.000,0.000,0.000,0.000,0.000,0.000
max,20232.000,3130327.000,9002.000,9018.000,200.000,29.000,400.000,288.000,127.000


In [232]:
# 중복 확인
store[store.duplicated()]

,기준_년분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,점포_수,유사_업종_점포_수,개업_율,개업_점포_수,폐업_률,폐업_점포_수,프랜차이즈_점포_수,서비스_업종_대분류


### 2-3-2. 분기별 데이터 연도 기준 변환

In [233]:
store_year = store[store['기준_년분기_코드'] == 20232].drop(['기준_년분기_코드', '상권_코드', '상권_구분_코드', '서비스_업종_코드', '서비스_업종_코드_명'], axis = 1)\
                                                    .groupby(['상권_구분_코드_명', '상권_코드_명', '서비스_업종_대분류'], as_index = False)\
                                                    .sum()

In [234]:
store_year.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4912 entries, 0 to 4911
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   상권_구분_코드_명  4912 non-null   object
 1   상권_코드_명     4912 non-null   object
 2   서비스_업종_대분류  4912 non-null   object
 3   점포_수        4912 non-null   int64 
 4   유사_업종_점포_수  4912 non-null   int64 
 5   개업_율        4912 non-null   int64 
 6   개업_점포_수     4912 non-null   int64 
 7   폐업_률        4912 non-null   int64 
 8   폐업_점포_수     4912 non-null   int64 
 9   프랜차이즈_점포_수  4912 non-null   int64 
dtypes: int64(7), object(3)
memory usage: 383.9+ KB


## 2-4. 직장인구 전처리

### 2-4-1. 변수 처리

In [235]:
working_popul = working_popul[['기준_년분기_코드', '상권_구분_코드_명', '상권_코드_명', '총_직장_인구_수']]

### 2-4-2. 결측치/이상치/중복

In [236]:
# 결측값 확인
working_popul.isnull().sum()

기준_년분기_코드     0
상권_구분_코드_명    0
상권_코드_명       0
총_직장_인구_수     0
dtype: int64

In [237]:
# 통계량 확인
working_popul.describe()

,기준_년분기_코드,총_직장_인구_수
count,6492.000,6492.000
mean,20227.500,1771.425
std,4.031,6246.210
min,20223.000,1.000
25%,20223.750,107.000
50%,20227.500,317.000
75%,20231.250,892.000
max,20232.000,104830.000


In [238]:
# 중복 확인
working_popul[working_popul.duplicated()]

,기준_년분기_코드,상권_구분_코드_명,상권_코드_명,총_직장_인구_수


### 2-4-3. 분기별 데이터 연도 기준 변환

In [239]:
working_popul_year = working_popul[working_popul['기준_년분기_코드'] == 20232].drop(['기준_년분기_코드'],axis=1)

In [240]:
working_popul_year.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1623 entries, 11546 to 25988
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   상권_구분_코드_명  1623 non-null   object
 1   상권_코드_명     1623 non-null   object
 2   총_직장_인구_수   1623 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 50.7+ KB


## 2-5. 집객시설 전처리

### 2-5-1. 변수 처리

In [241]:
# 파생변수 생성
facilitie['초중고_수'] = facilitie['초등학교_수']+facilitie['중학교_수']+facilitie['고등학교_수']

# 불필요한 변수 제거
facilitie = facilitie.drop(['집객시설_수','철도_역_수', '초등학교_수', '중학교_수', '고등학교_수'], axis=1)

### 2-5-2. 결측치/이상치/중복

In [242]:
# 결측값 확인
facilitie.isnull().sum()

기준_년분기_코드        0
상권_구분_코드_명       0
상권_코드_명          0
관공서_수         3792
은행_수          4364
종합병원_수        6220
일반_병원_수       5784
약국_수          2276
유치원_수         5708
대학교_수         5976
백화점_수         6268
슈퍼마켓_수        5892
극장_수          5976
숙박_시설_수       5732
공항_수          6308
버스_터미널_수      6304
지하철_역_수       5532
버스_정거장_수      1232
초중고_수         6308
dtype: int64

In [243]:
facilitie = facilitie.fillna(0)

In [244]:
# 통계량 확인
facilitie.describe()

,기준_년분기_코드,관공서_수,은행_수,종합병원_수,일반_병원_수,약국_수,유치원_수,대학교_수,백화점_수,슈퍼마켓_수,극장_수,숙박_시설_수,공항_수,버스_터미널_수,지하철_역_수,버스_정거장_수,초중고_수
count,6312.000,6312.000,6312.000,6312.000,6312.000,6312.000,6312.000,6312.000,6312.000,6312.000,6312.000,6312.000,6312.000,6312.000,6312.000,6312.000,6312.000
mean,20227.500,0.695,1.015,0.015,0.107,2.359,0.112,0.087,0.008,0.102,0.072,0.213,0.003,0.001,0.176,3.896,0.003
std,4.031,1.172,3.010,0.127,0.396,4.034,0.385,0.466,0.094,0.495,0.352,1.027,0.101,0.036,0.535,4.959,0.126
min,20223.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,20223.750,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000
50%,20227.500,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.000,0.000
75%,20231.250,1.000,1.000,0.000,0.000,3.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,5.000,0.000
max,20232.000,14.000,57.000,2.000,4.000,52.000,6.000,7.000,2.000,8.000,5.000,18.000,4.000,1.000,5.000,85.000,5.000


In [245]:
# 중복 확인
facilitie[facilitie.duplicated()]

,기준_년분기_코드,상권_구분_코드_명,상권_코드_명,관공서_수,은행_수,종합병원_수,일반_병원_수,약국_수,유치원_수,대학교_수,백화점_수,슈퍼마켓_수,극장_수,숙박_시설_수,공항_수,버스_터미널_수,지하철_역_수,버스_정거장_수,초중고_수


### 2-5-3. 분기별 데이터 연도 기준 변환

In [246]:
facilitie_year = facilitie[facilitie['기준_년분기_코드'] == 20232].drop(['기준_년분기_코드'], axis = 1)

In [247]:
facilitie_year.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1578 entries, 0 to 17356
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   상권_구분_코드_명  1578 non-null   object 
 1   상권_코드_명     1578 non-null   object 
 2   관공서_수       1578 non-null   float64
 3   은행_수        1578 non-null   float64
 4   종합병원_수      1578 non-null   float64
 5   일반_병원_수     1578 non-null   float64
 6   약국_수        1578 non-null   float64
 7   유치원_수       1578 non-null   float64
 8   대학교_수       1578 non-null   float64
 9   백화점_수       1578 non-null   float64
 10  슈퍼마켓_수      1578 non-null   float64
 11  극장_수        1578 non-null   float64
 12  숙박_시설_수     1578 non-null   float64
 13  공항_수        1578 non-null   float64
 14  버스_터미널_수    1578 non-null   float64
 15  지하철_역_수     1578 non-null   float64
 16  버스_정거장_수    1578 non-null   float64
 17  초중고_수       1578 non-null   float64
dtypes: float64(16), object(2)
memory usage: 234.2+ KB


In [248]:
facilitie_year.describe()

,관공서_수,은행_수,종합병원_수,일반_병원_수,약국_수,유치원_수,대학교_수,백화점_수,슈퍼마켓_수,극장_수,숙박_시설_수,공항_수,버스_터미널_수,지하철_역_수,버스_정거장_수,초중고_수
count,1578.000,1578.000,1578.000,1578.000,1578.000,1578.000,1578.000,1578.000,1578.000,1578.000,1578.000,1578.000,1578.000,1578.000,1578.000,1578.000
mean,0.695,1.015,0.015,0.107,2.359,0.112,0.087,0.008,0.102,0.072,0.213,0.003,0.001,0.176,3.896,0.003
std,1.172,3.010,0.127,0.396,4.035,0.386,0.466,0.094,0.495,0.352,1.027,0.101,0.036,0.536,4.960,0.126
min,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000
50%,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.000,0.000
75%,1.000,1.000,0.000,0.000,3.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,5.000,0.000
max,14.000,57.000,2.000,4.000,52.000,6.000,7.000,2.000,8.000,5.000,18.000,4.000,1.000,5.000,85.000,5.000


## 2-6. 추정매출 전처리

### 2-6-1. 변수 처리

In [249]:
# 파생변수 생성
estimated_sales['시간대_21~06_매출_금액'] = estimated_sales['시간대_00~06_매출_금액'] + estimated_sales['시간대_21~24_매출_금액']
estimated_sales['시간대_21~06_매출_건수'] = estimated_sales['시간대_건수~06_매출_건수'] + estimated_sales['시간대_건수~24_매출_건수']
estimated_sales['연령대_30_40_매출_금액'] = estimated_sales['연령대_30_매출_금액'] + estimated_sales['연령대_40_매출_금액']
estimated_sales['연령대_50_60_매출_금액'] = estimated_sales['연령대_50_매출_금액'] + estimated_sales['연령대_60_이상_매출_금액']
estimated_sales['연령대_30_40_매출_건수'] = estimated_sales['연령대_30_매출_건수'] + estimated_sales['연령대_40_매출_건수']
estimated_sales['연령대_50_60_매출_건수'] = estimated_sales['연령대_50_매출_건수'] + estimated_sales['연령대_60_이상_매출_건수']

estimated_sales['서비스_업종_대분류'] = estimated_sales['서비스_업종_코드'].str[:3]

# 불필요한 변수 제거
estimated_sales = estimated_sales.drop(columns = ['서비스_업종_코드'],
                                       axis = 1)

estimated_sales = estimated_sales.drop(columns = ['월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액',
                                                  '월요일_매출_건수', '화요일_매출_건수', '수요일_매출_건수', '목요일_매출_건수', '금요일_매출_건수', '토요일_매출_건수', '일요일_매출_건수'],
                                       axis = 1)
estimated_sales = estimated_sales.drop(columns = ['시간대_00~06_매출_금액', '시간대_21~24_매출_금액', '시간대_건수~06_매출_건수', '시간대_건수~24_매출_건수',
                                                 '연령대_30_매출_금액','연령대_40_매출_금액','연령대_50_매출_금액','연령대_60_이상_매출_금액',
                                                 '연령대_30_매출_건수','연령대_40_매출_건수','연령대_50_매출_건수','연령대_60_이상_매출_건수'],
                                       axis = 1)

# 변수 이름 변경
estimated_sales = estimated_sales.rename(columns = {'시간대_건수~11_매출_건수' : '시간대_06~11_매출_건수',
                                                         '시간대_건수~14_매출_건수' : '시간대_11~14_매출_건수',
                                                         '시간대_건수~17_매출_건수' : '시간대_14~17_매출_건수',
                                                         '시간대_건수~21_매출_건수' : '시간대_17~21_매출_건수'})



### 2-6-2. 결측치/이상치/중복

In [250]:
# 결측값 확인
estimated_sales.isnull().sum()

기준_년분기_코드          0
상권_구분_코드           0
상권_구분_코드_명         0
상권_코드              0
상권_코드_명            0
서비스_업종_코드_명        0
당월_매출_금액           0
당월_매출_건수           0
주중_매출_금액           0
주말_매출_금액           0
시간대_06~11_매출_금액    0
시간대_11~14_매출_금액    0
시간대_14~17_매출_금액    0
시간대_17~21_매출_금액    0
남성_매출_금액           0
여성_매출_금액           0
연령대_10_매출_금액       0
연령대_20_매출_금액       0
주중_매출_건수           0
주말_매출_건수           0
시간대_06~11_매출_건수    0
시간대_11~14_매출_건수    0
시간대_14~17_매출_건수    0
시간대_17~21_매출_건수    0
남성_매출_건수           0
여성_매출_건수           0
연령대_10_매출_건수       0
연령대_20_매출_건수       0
시간대_21~06_매출_금액    0
시간대_21~06_매출_건수    0
연령대_30_40_매출_금액    0
연령대_50_60_매출_금액    0
연령대_30_40_매출_건수    0
연령대_50_60_매출_건수    0
서비스_업종_대분류         0
dtype: int64

In [251]:
# 통계량 확인
estimated_sales.describe()

,기준_년분기_코드,상권_코드,당월_매출_금액,당월_매출_건수,주중_매출_금액,주말_매출_금액,시간대_06~11_매출_금액,시간대_11~14_매출_금액,시간대_14~17_매출_금액,시간대_17~21_매출_금액,남성_매출_금액,여성_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,주중_매출_건수,주말_매출_건수,시간대_06~11_매출_건수,시간대_11~14_매출_건수,시간대_14~17_매출_건수,시간대_17~21_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,시간대_21~06_매출_금액,시간대_21~06_매출_건수,연령대_30_40_매출_금액,연령대_50_60_매출_금액,연령대_30_40_매출_건수,연령대_50_60_매출_건수
count,84143.000,84143.000,84143.000,84143.000,84143.000,84143.000,84143.000,84143.000,84143.000,84143.000,84143.000,84143.000,84143.000,84143.000,84143.000,84143.000,84143.000,84143.000,84143.000,84143.000,84143.000,84143.000,84143.000,84143.000,84143.000,84143.000,84143.000,84143.000,84143.000,84143.000
mean,20227.506,3115479.183,1116599271.468,39184.849,842969184.583,273630043.386,116413408.602,286186909.051,280424841.016,306646276.190,506115717.968,483588097.551,5734552.208,135382675.504,29499.636,9685.213,5416.564,10204.191,8307.715,10443.374,19337.490,18168.507,577.372,7898.857,126927760.944,4813.006,431464849.292,417119530.685,16555.601,12474.165
std,4.031,14347.760,9371981938.469,168454.273,6458423614.170,3509552781.251,1172352096.256,2477461950.285,3124214644.465,2771104568.472,4449083121.756,3616311150.276,31894990.608,909570386.425,117929.298,57711.259,29263.631,45978.917,44186.686,47735.342,83502.247,83244.346,3617.461,42063.066,1000325381.326,23992.316,3469029992.731,3773146796.252,73433.475,61385.401
min,20223.000,3001491.000,12.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,20224.000,3110547.000,48560077.000,868.000,36205605.500,7456979.500,145574.500,5491691.000,7869005.500,11816467.500,19597595.500,18111528.500,0.000,1369387.500,654.000,153.000,7.000,109.000,173.000,226.000,390.000,342.000,0.000,43.000,0.000,0.000,15857293.000,15467433.000,324.000,297.000
50%,20231.000,3111085.000,188164542.000,4646.000,140846234.000,38302621.000,6268042.000,34963031.000,36234148.000,50583967.000,84295031.000,75951653.000,166368.000,12428462.000,3446.000,1036.000,169.000,985.000,983.000,1304.000,2286.000,1968.000,12.000,446.000,4061538.000,77.000,69746034.000,63905807.000,1783.000,1642.000
75%,20232.000,3120185.000,686193977.000,25492.500,520555159.000,150576100.000,48504462.500,153603359.000,143040793.500,186412033.000,314344883.500,290361639.500,2273891.500,64077848.500,19055.500,5839.500,1996.000,6405.500,5206.500,6992.500,12096.000,11579.000,174.000,3306.000,51155789.000,1497.000,260456794.500,241524127.500,9897.000,8588.000
max,20232.000,3130327.000,995677000000.000,12090273.000,669799000000.000,485357000000.000,107916000000.000,242249000000.000,326543000000.000,386343000000.000,537243000000.000,304162000000.000,1549259735.000,90669553834.000,5827863.000,6262410.000,1474148.000,2611826.000,3245770.000,4287338.000,6708993.000,4615812.000,329040.000,3087414.000,126314287024.000,1514884.000,354347000000.000,411544000000.000,4519958.000,5329793.000


In [252]:
# 중복 확인
estimated_sales[estimated_sales.duplicated()]

,기준_년분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드_명,당월_매출_금액,당월_매출_건수,주중_매출_금액,주말_매출_금액,시간대_06~11_매출_금액,시간대_11~14_매출_금액,시간대_14~17_매출_금액,시간대_17~21_매출_금액,남성_매출_금액,여성_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,주중_매출_건수,주말_매출_건수,시간대_06~11_매출_건수,시간대_11~14_매출_건수,시간대_14~17_매출_건수,시간대_17~21_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,시간대_21~06_매출_금액,시간대_21~06_매출_건수,연령대_30_40_매출_금액,연령대_50_60_매출_금액,연령대_30_40_매출_건수,연령대_50_60_매출_건수,서비스_업종_대분류


In [253]:
def sales_tran(df):
    if df['당월_법인_매출_금액'] < 0:
        return df['당월_매출_금액'] - df['당월_법인_매출_금액']
    else :
        return df['당월_매출_금액']
estimated_sales = estimated_sales[estimated_sales['당월_매출_금액'] > estimated_sales['당월_매출_금액'].quantile(0.05)]
estimated_sales['당월_개인_매출_금액'] = estimated_sales['남성_매출_금액'] + estimated_sales['여성_매출_금액']
estimated_sales['당월_법인_매출_금액'] =  estimated_sales['당월_매출_금액'] - estimated_sales['당월_개인_매출_금액']

estimated_sales['당월_매출_금액'] = estimated_sales.apply(lambda x : sales_tran(x) , axis=1)

estimated_sales.loc[estimated_sales['당월_법인_매출_금액']<0, '당월_법인_매출_금액'] = 0

In [254]:
estimated_sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 79910 entries, 119954 to 204096
Data columns (total 37 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   기준_년분기_코드        79910 non-null  int64  
 1   상권_구분_코드         79910 non-null  object 
 2   상권_구분_코드_명       79910 non-null  object 
 3   상권_코드            79910 non-null  int64  
 4   상권_코드_명          79910 non-null  object 
 5   서비스_업종_코드_명      79910 non-null  object 
 6   당월_매출_금액         79910 non-null  float64
 7   당월_매출_건수         79910 non-null  int64  
 8   주중_매출_금액         79910 non-null  float64
 9   주말_매출_금액         79910 non-null  float64
 10  시간대_06~11_매출_금액  79910 non-null  float64
 11  시간대_11~14_매출_금액  79910 non-null  float64
 12  시간대_14~17_매출_금액  79910 non-null  float64
 13  시간대_17~21_매출_금액  79910 non-null  float64
 14  남성_매출_금액         79910 non-null  float64
 15  여성_매출_금액         79910 non-null  float64
 16  연령대_10_매출_금액     79910 non-null  int64  
 17  연령대_20_매출_금

### 2-6-3. 분기별 데이터 연도 기준 변환

In [255]:
estimated_sales_year = estimated_sales.drop(columns = ['기준_년분기_코드','상권_구분_코드','상권_코드', '서비스_업종_코드_명'])\
                                      .groupby(['상권_구분_코드_명', '상권_코드_명', '서비스_업종_대분류'])\
                                      .mean()\
                                      .reset_index()

In [256]:
estimated_sales_year.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4132 entries, 0 to 4131
Data columns (total 33 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   상권_구분_코드_명       4132 non-null   object 
 1   상권_코드_명          4132 non-null   object 
 2   서비스_업종_대분류       4132 non-null   object 
 3   당월_매출_금액         4132 non-null   float64
 4   당월_매출_건수         4132 non-null   float64
 5   주중_매출_금액         4132 non-null   float64
 6   주말_매출_금액         4132 non-null   float64
 7   시간대_06~11_매출_금액  4132 non-null   float64
 8   시간대_11~14_매출_금액  4132 non-null   float64
 9   시간대_14~17_매출_금액  4132 non-null   float64
 10  시간대_17~21_매출_금액  4132 non-null   float64
 11  남성_매출_금액         4132 non-null   float64
 12  여성_매출_금액         4132 non-null   float64
 13  연령대_10_매출_금액     4132 non-null   float64
 14  연령대_20_매출_금액     4132 non-null   float64
 15  주중_매출_건수         4132 non-null   float64
 16  주말_매출_건수         4132 non-null   float64
 17  시간대_06~11_매출_건

## 2-7. 데이터 병합

In [257]:
df = pd.merge(living_popul_year, resident_popul_year)
df = pd.merge(df, store_year)
df = pd.merge(df, working_popul_year)
df = pd.merge(df, facilitie_year)
df = pd.merge(df, estimated_sales_year)

In [258]:
df.loc[df['서비스_업종_대분류'] == "CS1", '서비스_업종_대분류'] = '요식업'
df.loc[df['서비스_업종_대분류'] == "CS2", '서비스_업종_대분류'] = '서비스업'
df.loc[df['서비스_업종_대분류'] == "CS3", '서비스_업종_대분류'] = '도매 및 소매업'

In [259]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3923 entries, 0 to 3922
Data columns (total 78 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   상권_구분_코드_명        3923 non-null   object 
 1   상권_코드_명           3923 non-null   object 
 2   총_유동인구_수          3923 non-null   float64
 3   남성_유동인구_수         3923 non-null   float64
 4   여성_유동인구_수         3923 non-null   float64
 5   연령대_10_유동인구_수     3923 non-null   float64
 6   연령대_20_유동인구_수     3923 non-null   float64
 7   시간대_06_11_유동인구_수  3923 non-null   float64
 8   시간대_11_14_유동인구_수  3923 non-null   float64
 9   시간대_14_17_유동인구_수  3923 non-null   float64
 10  시간대_17_21_유동인구_수  3923 non-null   float64
 11  시간대_21_06_유동인구_수  3923 non-null   float64
 12  주중_유동인구_수         3923 non-null   float64
 13  주말_유동인구_수         3923 non-null   float64
 14  연령대_30_40_유동인구_수  3923 non-null   float64
 15  연령대_50_60_유동인구_수  3923 non-null   float64
 16  총_상주인구_수          3923 non-null   int64  


In [260]:
df.describe()

,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,연령대_10_유동인구_수,연령대_20_유동인구_수,시간대_06_11_유동인구_수,시간대_11_14_유동인구_수,시간대_14_17_유동인구_수,시간대_17_21_유동인구_수,시간대_21_06_유동인구_수,주중_유동인구_수,주말_유동인구_수,연령대_30_40_유동인구_수,연령대_50_60_유동인구_수,총_상주인구_수,남성_상주인구_수,여성_상주인구_수,연령대_10_상주인구_수,연령대_20_상주인구_수,연령대_30_40_상주인구_수,연령대_50_60_상주인구_수,점포_수,유사_업종_점포_수,개업_율,개업_점포_수,폐업_률,폐업_점포_수,프랜차이즈_점포_수,총_직장_인구_수,관공서_수,은행_수,종합병원_수,일반_병원_수,약국_수,유치원_수,대학교_수,백화점_수,슈퍼마켓_수,극장_수,숙박_시설_수,공항_수,버스_터미널_수,지하철_역_수,버스_정거장_수,초중고_수,당월_매출_금액,당월_매출_건수,주중_매출_금액,주말_매출_금액,시간대_06~11_매출_금액,시간대_11~14_매출_금액,시간대_14~17_매출_금액,시간대_17~21_매출_금액,남성_매출_금액,여성_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,주중_매출_건수,주말_매출_건수,시간대_06~11_매출_건수,시간대_11~14_매출_건수,시간대_14~17_매출_건수,시간대_17~21_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,시간대_21~06_매출_금액,시간대_21~06_매출_건수,연령대_30_40_매출_금액,연령대_50_60_매출_금액,연령대_30_40_매출_건수,연령대_50_60_매출_건수,당월_개인_매출_금액,당월_법인_매출_금액
count,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000,3923.000
mean,990662.923,471951.017,518711.919,126706.627,177990.600,201128.885,127850.164,129232.237,172422.284,360029.433,714476.252,276186.753,335048.336,350917.435,2750.341,1362.167,1388.174,260.154,299.233,943.378,1247.577,110.426,118.797,43.844,2.858,42.576,3.112,8.371,2081.319,0.789,1.185,0.017,0.127,2.767,0.128,0.090,0.008,0.096,0.081,0.238,0.000,0.002,0.196,4.347,0.004,716658818.316,28194.005,544676659.106,171982131.686,76625819.809,183894450.087,167653034.862,200391532.903,336735886.689,303762313.096,3648645.657,82712757.852,21170.304,7023.701,3864.181,7179.614,5722.842,7621.760,14430.050,12671.629,414.505,5452.889,88093943.676,3805.609,270131484.319,284004767.417,11690.293,9544.004,640498199.785,76160618.531
std,932634.453,454400.472,482522.101,116934.900,232061.831,185262.826,141562.786,144795.256,175292.158,331467.574,685232.623,254389.107,347161.178,318155.460,2443.832,1208.957,1240.488,269.158,276.678,891.059,1097.924,284.342,292.492,49.811,5.154,51.772,5.955,17.833,6871.566,1.241,3.267,0.135,0.429,4.304,0.413,0.469,0.099,0.505,0.375,1.111,0.000,0.039,0.563,5.211,0.138,2174786763.807,52068.062,1657272497.681,614689342.021,373350007.471,668927621.661,697568241.978,517324334.397,1013487229.373,917962999.583,11267898.599,234556263.561,38451.902,14973.397,8391.824,15675.837,12534.010,13864.152,25397.811,25383.649,1188.485,13289.973,218474280.247,7621.159,718031359.241,1120052415.546,21595.427,21054.942,1889246998.319,451482268.354
min,41356.500,18580.500,18881.000,1763.500,2419.500,7723.000,5020.250,4708.500,5842.000,5784.250,29058.000,9312.000,9347.500,11240.750,1.000,0.000,0.000,0.000,0.000,0.000,1.000,3.000,3.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,6102740.000,5.500,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2411357.750,0.000
25%,369325.000,176654.250,192909.625,43398.500,51955.000,75427.750,47155.250,47247.125,63211.500,123050.500,267541.500,102081.000,116936.250,129015.125,969.000,487.000,489.000,77.000,97.000,324.000,438.000,30.000,33.000,0.000,0.000,0.000,1.000,1.000,158.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,142839539.071,3544.405,108274069.548,29395039.417,5412530.756,27120231.896,25933295.268,40101665.375,69544279.708,54203932.458,215657.875,8720095.25

In [261]:
df.head()

,상권_구분_코드_명,상권_코드_명,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,연령대_10_유동인구_수,연령대_20_유동인구_수,시간대_06_11_유동인구_수,시간대_11_14_유동인구_수,시간대_14_17_유동인구_수,시간대_17_21_유동인구_수,시간대_21_06_유동인구_수,주중_유동인구_수,주말_유동인구_수,연령대_30_40_유동인구_수,연령대_50_60_유동인구_수,총_상주인구_수,남성_상주인구_수,여성_상주인구_수,연령대_10_상주인구_수,연령대_20_상주인구_수,연령대_30_40_상주인구_수,연령대_50_60_상주인구_수,서비스_업종_대분류,점포_수,유사_업종_점포_수,개업_율,개업_점포_수,폐업_률,폐업_점포_수,프랜차이즈_점포_수,총_직장_인구_수,관공서_수,은행_수,종합병원_수,일반_병원_수,약국_수,유치원_수,대학교_수,백화점_수,슈퍼마켓_수,극장_수,숙박_시설_수,공항_수,버스_터미널_수,지하철_역_수,버스_정거장_수,초중고_수,당월_매출_금액,당월_매출_건수,주중_매출_금액,주말_매출_금액,시간대_06~11_매출_금액,시간대_11~14_매출_금액,시간대_14~17_매출_금액,시간대_17~21_매출_금액,남성_매출_금액,여성_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,주중_매출_건수,주말_매출_건수,시간대_06~11_매출_건수,시간대_11~14_매출_건수,시간대_14~17_매출_건수,시간대_17~21_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,시간대_21~06_매출_금액,시간대_21~06_매출_건수,연령대_30_40_매출_금액,연령대_50_60_매출_금액,연령대_30_40_매출_건수,연령대_50_60_매출_건수,당월_개인_매출_금액,당월_법인_매출_금액
0,골목상권,4.19민주묘지역 2번,868030.000,361490.250,506539.250,128381.000,163223.250,169905.000,105148.000,103902.500,145960.750,343114.000,619012.250,249017.500,221593.250,354833.000,1247,596,651,135,121,324,667,요식업,38,48,8,2,4,1,10,96,0.000,0.000,0.000,0.000,1.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,411361983.913,20467.522,249668438.609,161693545.304,15936808.130,104889853.435,75817345.826,160273338.130,223277977.609,169226593.739,2611829.304,44509673.870,13399.130,7068.391,1599.739,5434.174,4205.000,6571.000,10135.130,10052.696,309.348,4043.435,54444638.391,2657.609,135490587.522,209892480.783,7626.870,8208.043,392504571.348,18857412.565
1,골목상권,4.19민주묘지역 2번,868030.000,361490.250,506539.250,128381.000,163223.250,169905.000,105148.000,103902.500,145960.750,343114.000,619012.250,249017.500,221593.250,354833.000,1247,596,651,135,121,324,667,서비스업,17,17,0,0,0,0,0,96,0.000,0.000,0.000,0.000,1.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,34384167.000,1306.500,21903351.000,12480816.000,862554.500,9952889.750,10859155.000,12330743.000,20974955.000,13117808.250,136012.500,3947675.500,824.500,482.000,55.000,368.500,438.250,440.250,919.500,384.000,10.500,177.500,378824.750,4.500,20604973.000,9404102.250,788.750,326.500,34092763.250,291403.750
2,골목상권,4.19민주묘지역 2번,868030.000,361490.250,506539.250,128381.000,163223.250,169905.000,105148.000,103902.500,145960.750,343114.000,619012.250,249017.500,221593.250,354833.000,1247,596,651,135,121,324,667,도매 및 소매업,20,23,33,1,50,1,3,96,0.000,0.000,0.000,0.000,1.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,304908427.000,44877.000,212658965.000,92249462.000,45252762.000,30557113.000,39478507.000,86471437.000,191164903.000,113743524.000,557091.000,62379154.000,31545.000,13332.000,7512.000,5310.000,6319.000,11742.000,28457.000,16420.000,165.000,10557.000,103148608.000,13994.000,131357183.000,110614999.000,18677.000,15478.000,304908427.000,0.000
3,골목상권,GS강동자이아파트,839601.250,383011.750,456589.750,175441.750,89926.500,169725.750,98596.500,101511.000,145801.500,323967.250,576675.250,262926.250,235045.000,339189.000,3443,1725,1718,404,413,1066,1560,요식업,32,42,150,4,31,3,10,411,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.000,0.000,286289020.000,17363.625,185667042.375,100621977.625,14879907.250,53871970.812,47333706.000,108390689.188,153030456.062,114711833.625,1023351.062,31041859.250,11307.812,6055.812,1677.688,4133.750,4211.875,5266.438,8529.938,8210.875,87.625,2876.000,61812746.750,2073.875,125058005.500,110619074.062,8390.500,5385.375,267742289.688,18546730.312
4,골목상권,GS강동자이아파트,839601.250,383011.750,456589.750,175441.750,89926.500,169725.750,98596.500,101511.000,145801.500,323967.250,576675.250,262926.250,235045.000,339189.000,3443,1725,1718,404,413,1066,1560,서비스업,71,75,133,4,7,1,4,411,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.000,0.000,107339195.182,1676.727,96099351.455,11239843.727,3162804.364,13415256.318,39256329.091,46620649.727,37947121.136,52851738.591,214350.318,1305563.318,1230.318,446

In [262]:
df.to_csv("final_df.csv")

# 3. 모델 학습

## 3-1. 분석에 적합한 데이터로 변환

In [263]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

label_df = df.drop(columns = ['상권_구분_코드_명', '상권_코드_명'])
label_df_ID = df[['상권_구분_코드_명', '상권_코드_명']]

scaler = StandardScaler()
col_num = list(label_df.select_dtypes(include = 'number').columns)
label_df[col_num] = scaler.fit_transform(label_df[col_num])

col_cat = list(label_df.select_dtypes(include = 'object').columns)
le = LabelEncoder()
label_df[col_cat] = label_df[col_cat].apply(le.fit_transform)

In [264]:
label_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3923 entries, 0 to 3922
Data columns (total 76 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   총_유동인구_수          3923 non-null   float64
 1   남성_유동인구_수         3923 non-null   float64
 2   여성_유동인구_수         3923 non-null   float64
 3   연령대_10_유동인구_수     3923 non-null   float64
 4   연령대_20_유동인구_수     3923 non-null   float64
 5   시간대_06_11_유동인구_수  3923 non-null   float64
 6   시간대_11_14_유동인구_수  3923 non-null   float64
 7   시간대_14_17_유동인구_수  3923 non-null   float64
 8   시간대_17_21_유동인구_수  3923 non-null   float64
 9   시간대_21_06_유동인구_수  3923 non-null   float64
 10  주중_유동인구_수         3923 non-null   float64
 11  주말_유동인구_수         3923 non-null   float64
 12  연령대_30_40_유동인구_수  3923 non-null   float64
 13  연령대_50_60_유동인구_수  3923 non-null   float64
 14  총_상주인구_수          3923 non-null   float64
 15  남성_상주인구_수         3923 non-null   float64
 16  여성_상주인구_수         3923 non-null   float64


In [285]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_blobs

# DBSCAN 모델 생성 및 학습
dbscan = DBSCAN(eps=5, min_samples=15)
labels = dbscan.fit_predict(label_df)

label_df['Cluster_Label'] = labels

label_df['Cluster_Label'].value_counts()

Cluster_Label
 0    3423
-1     484
 1      16
Name: count, dtype: int64